**Use**: Identifies the top-k hyper-parameters that yield the highest Auto-Labeling coverage (Coverage-Mean), while ensuring that the coverage remains within the error threshold defined by Auto-Labeling error (eps). 

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import itertools
sys.path.append('../')
sys.path.append('../../')

In [2]:
# Options
root_pfx = "../../../outputs/"
file_identifier = "mnist_lenet_calib_eval_hyp_2_squentropy_tbal_eval_hyp/tbal_post_hoc_search_mnist_lenet_calib_eval_hyp_2_squentropy_tbal_eval_hyp__01-22-2024__18-04-37"
eps_percent = 0.05 * 100


# Read xlsx file
df = pd.read_excel(f"{root_pfx}{file_identifier}.xlsx", sheet_name=0)
df = df.drop(columns=['Unnamed: 0'])

Filter records in dataframe

In [3]:
# Retain all records with Auto-Labeling-Err-Mean <= eps
df_1 = df.query("`Auto-Labeling-Err-Mean` <= @eps_percent")

# Sort by col: Coverage-Mean in descending order, and then by col: calib_conf in ascending order
df_1 = df_1.sort_values(["Coverage-Mean", "calib_conf"], ascending = [False, True])

# Change all calib_conf with NaN to "None"
df_1['calib_conf'] = df['calib_conf'].fillna("None")

# Sanity checks
assert df_1.query("`Auto-Labeling-Err-Mean` > @eps_percent").empty

In [4]:
df_1

,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,...,training_conf.max_epochs,training_conf.num_bins,training_conf.optimizer,training_conf.reg,training_conf.weight_decay,training_conf_g.batch_size,training_conf_g.max_epochs,training_conf_g.optimizer,training_conf_g.weight_decay,weight_decay
43,auto_label_opt_v0,fmfp,0.25,500,500,500,3.2628,85.72,77.6992,1.3161,...,NaN,NaN,NaN,NaN,NaN,64.0,500.0,adam,1.00,0.001
39,auto_label_opt_v0,fmfp,0.25,500,500,500,3.2173,85.68,77.6826,1.3924,...,NaN,NaN,NaN,NaN,NaN,64.0,500.0,adam,0.10,0.001
18,auto_label_opt_v0,fmfp,0.25,500,500,500,3.3343,85.36,77.0983,0.6612,...,NaN,NaN,NaN,NaN,NaN,64.0,500.0,adam,0.01,0.001
26,auto_label_opt_v0,fmfp,0.25,500,500,500,3.0970,85.28,77.1104,1.7402,...,NaN,NaN,NaN,NaN,NaN,64.0,500.0,adam,1.00,0.001
24,auto_label_opt_v0,fmfp,0.25,500,500,500,2.5737,85.12,77.6468,0.8571,...,NaN,NaN,NaN,NaN,NaN,64.0,500.0,adam,1.00,0.001
12,auto_label_opt_v0,fmfp,0.25,500,500,500,2.2335,84.84,77.1897,1.8457,...,NaN,NaN,NaN,NaN,NaN,64.0,500.0,adam,0.10,0.001
22,auto_label_opt_v0,fmfp,0.25,500,500,500,2.7408,84.84,77.0988,1.7687,...,NaN,NaN,NaN,NaN,NaN,64.0,500.0,adam,0.10,0.001
30,auto_label_opt_v0,fmfp,0.25,500,500,500,2.2203,84.80,76.9518,1.4361,...,NaN,NaN,NaN,NaN,NaN,64.0,500.0,adam,0.10,0.001
34,auto_label_opt_v0,fmfp,0.25,500,500,500,1.9921,84.60,76.9523,1.5073,...,NaN,NaN,NaN,NaN,NaN,64.0,500.0,adam,1.00,0.001
17,auto_label_opt_v0,fmfp,0.25,500,500,500,1.8959,84.52,76.9575,1.8567,...,NaN,NaN,NaN,NaN,NaN,64.0,500.0,adam,1.00,0.001


In [5]:
# Set display option to show all columns
pd.set_option('display.max_columns', None)

train_time_methods = ['std_cross_entropy', 'crl', 'fmfp', 'squentropy']
post_hoc_methods = ['None','auto_label_opt_v0','temp_scaling','dirichlet','scaling_binning','histogram_binning_top_label'] 

def print_top_k_post_hoc_methods(top_k):
    for tm, phm in itertools.product(train_time_methods, post_hoc_methods):
        df_filtered = df_1[df_1['calib_conf'] == phm]
        df_filtered2 = df_filtered[df_filtered['training_conf'] == tm]
        if top_k == 1:
            # Retain ties for top-1
            max_value = df_filtered2['Coverage-Mean'].max()
            df_3 = df_filtered2[df_filtered2['Coverage-Mean'] == max_value]
        else:
            df_3 = df_filtered2.head(top_k)
        display(df_3)

In [6]:
# top k occurences for each method
top_k = 1 

print_top_k_post_hoc_methods(top_k)

,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,Coverage-Std,Avg-ECE-Val-Std,alpha_1,batch_size,calib_val_frac,class_wise,eps,features_key,l1,l2,l3,learning_rate,lr_g,max_epochs,method,model_conf,momentum,num_runs,optimizer,points_per_bin,query_batch_frac,regularize,seed_frac,training_conf.batch_size,training_conf.learning_rate,training_conf.max_epochs,training_conf.num_bins,training_conf.optimizer,training_conf.reg,training_conf.weight_decay,training_conf_g.batch_size,training_conf_g.max_epochs,training_conf_g.optimizer,training_conf_g.weight_decay,weight_decay


,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,Coverage-Std,Avg-ECE-Val-Std,alpha_1,batch_size,calib_val_frac,class_wise,eps,features_key,l1,l2,l3,learning_rate,lr_g,max_epochs,method,model_conf,momentum,num_runs,optimizer,points_per_bin,query_batch_frac,regularize,seed_frac,training_conf.batch_size,training_conf.learning_rate,training_conf.max_epochs,training_conf.num_bins,training_conf.optimizer,training_conf.reg,training_conf.weight_decay,training_conf_g.batch_size,training_conf_g.max_epochs,training_conf_g.optimizer,training_conf_g.weight_decay,weight_decay


,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,Coverage-Std,Avg-ECE-Val-Std,alpha_1,batch_size,calib_val_frac,class_wise,eps,features_key,l1,l2,l3,learning_rate,lr_g,max_epochs,method,model_conf,momentum,num_runs,optimizer,points_per_bin,query_batch_frac,regularize,seed_frac,training_conf.batch_size,training_conf.learning_rate,training_conf.max_epochs,training_conf.num_bins,training_conf.optimizer,training_conf.reg,training_conf.weight_decay,training_conf_g.batch_size,training_conf_g.max_epochs,training_conf_g.optimizer,training_conf_g.weight_decay,weight_decay


,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,Coverage-Std,Avg-ECE-Val-Std,alpha_1,batch_size,calib_val_frac,class_wise,eps,features_key,l1,l2,l3,learning_rate,lr_g,max_epochs,method,model_conf,momentum,num_runs,optimizer,points_per_bin,query_batch_frac,regularize,seed_frac,training_conf.batch_size,training_conf.learning_rate,training_conf.max_epochs,training_conf.num_bins,training_conf.optimizer,training_conf.reg,training_conf.weight_decay,training_conf_g.batch_size,training_conf_g.max_epochs,training_conf_g.optimizer,training_conf_g.weight_decay,weight_decay


,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,Coverage-Std,Avg-ECE-Val-Std,alpha_1,batch_size,calib_val_frac,class_wise,eps,features_key,l1,l2,l3,learning_rate,lr_g,max_epochs,method,model_conf,momentum,num_runs,optimizer,points_per_bin,query_batch_frac,regularize,seed_frac,training_conf.batch_size,training_conf.learning_rate,training_conf.max_epochs,training_conf.num_bins,training_conf.optimizer,training_conf.reg,training_conf.weight_decay,training_conf_g.batch_size,training_conf_g.max_epochs,training_conf_g.optimizer,training_conf_g.weight_decay,weight_decay


...

,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,Coverage-Std,Avg-ECE-Val-Std,alpha_1,batch_size,calib_val_frac,class_wise,eps,features_key,l1,l2,l3,learning_rate,lr_g,max_epochs,method,model_conf,momentum,num_runs,optimizer,points_per_bin,query_batch_frac,regularize,seed_frac,training_conf.batch_size,training_conf.learning_rate,training_conf.max_epochs,training_conf.num_bins,training_conf.optimizer,training_conf.reg,training_conf.weight_decay,training_conf_g.batch_size,training_conf_g.max_epochs,training_conf_g.optimizer,training_conf_g.weight_decay,weight_decay


,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,Coverage-Std,Avg-ECE-Val-Std,alpha_1,batch_size,calib_val_frac,class_wise,eps,features_key,l1,l2,l3,learning_rate,lr_g,max_epochs,method,model_conf,momentum,num_runs,optimizer,points_per_bin,query_batch_frac,regularize,seed_frac,training_conf.batch_size,training_conf.learning_rate,training_conf.max_epochs,training_conf.num_bins,training_conf.optimizer,training_conf.reg,training_conf.weight_decay,training_conf_g.batch_size,training_conf_g.max_epochs,training_conf_g.optimizer,training_conf_g.weight_decay,weight_decay


,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,Coverage-Std,Avg-ECE-Val-Std,alpha_1,batch_size,calib_val_frac,class_wise,eps,features_key,l1,l2,l3,learning_rate,lr_g,max_epochs,method,model_conf,momentum,num_runs,optimizer,points_per_bin,query_batch_frac,regularize,seed_frac,training_conf.batch_size,training_conf.learning_rate,training_conf.max_epochs,training_conf.num_bins,training_conf.optimizer,training_conf.reg,training_conf.weight_decay,training_conf_g.batch_size,training_conf_g.max_epochs,training_conf_g.optimizer,training_conf_g.weight_decay,weight_decay


,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,Coverage-Std,Avg-ECE-Val-Std,alpha_1,batch_size,calib_val_frac,class_wise,eps,features_key,l1,l2,l3,learning_rate,lr_g,max_epochs,method,model_conf,momentum,num_runs,optimizer,points_per_bin,query_batch_frac,regularize,seed_frac,training_conf.batch_size,training_conf.learning_rate,training_conf.max_epochs,training_conf.num_bins,training_conf.optimizer,training_conf.reg,training_conf.weight_decay,training_conf_g.batch_size,training_conf_g.max_epochs,training_conf_g.optimizer,training_conf_g.weight_decay,weight_decay


,calib_conf,training_conf,C_1,N_t,N_v,N_hyp_v,Auto-Labeling-Err-Mean,Coverage-Mean,Avg-ECE-Val-Mean,Auto-Labeling-Err-Std,Coverage-Std,Avg-ECE-Val-Std,alpha_1,batch_size,calib_val_frac,class_wise,eps,features_key,l1,l2,l3,learning_rate,lr_g,max_epochs,method,model_conf,momentum,num_runs,optimizer,points_per_bin,query_batch_frac,regularize,seed_frac,training_conf.batch_size,training_conf.learning_rate,training_conf.max_epochs,training_conf.num_bins,training_conf.optimizer,training_conf.reg,training_conf.weight_decay,training_conf_g.batch_size,training_conf_g.max_epochs,training_conf_g.optimizer,training_conf_g.weight_decay,weight_decay
